In [73]:
# %%writefile model_2.py
import numpy as np
import pandas as pd
from pymongo import MongoClient
import pprint
import string
import re
import datetime
import copy

from my_tools import get_bill_data

import statsmodels.api as sm
import scipy.stats as scs
from sklearn.linear_model import LogisticRegression

from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import normalize
from sklearn.metrics import recall_score, precision_score, accuracy_score, confusion_matrix


from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB#, ComplementNB unreleased as of 12/14

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
data = get_bill_data()

------------------
Creating column 'labels'...


/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


------------------
------------------
Data is from the 110th Congress (2007) to present
------------------


In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51337 entries, 0 to 51336
Data columns (total 22 columns):
index                51337 non-null int64
_id                  51337 non-null object
bill_status          51337 non-null object
body                 51337 non-null object
committee            50852 non-null object
congress_id          51337 non-null object
cosponsors           0 non-null object
cosponsors_url       36523 non-null object
desc                 51337 non-null object
intro_date           51337 non-null datetime64[ns]
leg_id               51337 non-null object
leg_type             51337 non-null object
leg_url              51337 non-null object
num_of_cosponsors    51337 non-null object
sponsor              51334 non-null object
sponsor_district     33823 non-null object
sponsor_party        51334 non-null object
sponsor_state        51334 non-null object
bill_char_counts     51337 non-null int64
intro_month          51337 non-null int64
session              51337 non

In [42]:
for i in data['num_of_cosponsors'].unique():
    print(i)

3
2
1
23
5
0
51
8
10
26
52
7
12
17
45
13
35
40
28
9
4
33
19
25
61
6
228
191
16
24
11
300
291
29
71
106
14
31
30
43
96
227
18
290
20
150
44
15
95
21
94
313
82
74
92
229
58
274
249
287
67
54
83
64
36
298
84
224
122
152
303
198
205
260
S. Rept. 110-184
60
22
77
50
109
46
113
171
63
296
32
348
56
123
48
39
53
37
332
27
305
301
69
308
294
100
364
65
55
116
269
34
273
66
145
129
87
72
262
41
79
101
314
49
163
76
283
235
309
86
118
73
80
156
59
170
343
154
178
302
125
115
128
304
140
TXT
70
75
90
121
385
62
174
188
159
81
68
147
120
99
238
378
47
299
107
104
185
38
312
326
127
89
292
103
78
97
114
136
42
176
102
172
93
91
202
126
226
236
167
173
134
112
144
105
199
157
130
57
277
119
165
141
110
111
241
138
85
168
244
98
255
88
142
208
135
263
146
221
276
297
151
184
139
132
160
158
148
161
220
155
248
182
153
253
240
325
133
230
246
131
149
175
164
315
344
233
117
286
280
192
162
216
223
245
209
143
215
108
183
206
406
352
211
217
222
204
124
181
All Actions
179
187
432
310
213
197
137
333
1

In [ ]:
# non-numeric num_of_cosponsors: S. Rept. 110-184, TXT, All Actions

In [49]:
data = data[(data['num_of_cosponsors'] != 'S. Rept. 110-184') &
           (data['num_of_cosponsors'] != 'TXT') &
           (data['num_of_cosponsors'] != 'All Actions')]

In [50]:
data['num_of_cosponsors'] = data['num_of_cosponsors'].apply(pd.to_numeric)

In [51]:
data.iloc[:5, 10:]

,leg_id,leg_type,leg_url,num_of_cosponsors,sponsor,sponsor_district,sponsor_party,sponsor_state,bill_char_counts,intro_month,session,labels
0,H R 6477,LAW,https://www.congress.gov/bill/114th-congress/h...,3,"Rep. Chabot, Steve",1,R,OH,5366,12,2,1
1,H R 6452,LAW,https://www.congress.gov/bill/114th-congress/h...,2,"Rep. Radewagen, Aumua Amata Coleman",At Large,R,AS,68574,12,2,1
2,H R 6451,LAW,https://www.congress.gov/bill/114th-congress/h...,1,"Rep. Denham, Jeff",10,R,CA,24935,12,2,1
3,H R 6450,LAW,https://www.congress.gov/bill/114th-congress/h...,2,"Rep. Chaffetz, Jason",3,R,UT,33853,12,2,1
4,H R 6431,LAW,https://www.congress.gov/bill/114th-congress/h...,23,"Rep. Kuster, Ann M.",2,D,NH,3465,12,2,1


In [76]:
data_regres = data.loc[:, ['congress_id', 
                          'num_of_cosponsors', 
                          'sponsor_party', 
                          'sponsor_state', 
                          'bill_char_counts', 
                          'intro_month', 
                          'session', 
                           'labels']]

In [77]:
# get dummies for congress_id, intro_month, sponsor_party, sponsor_state, session
data_dumm = pd.get_dummies(data_regres, columns = ['congress_id', 
                                            'intro_month', 
                                            'sponsor_party', 
                                            'sponsor_state', 
                                            'session'], 
                           drop_first=True)

In [78]:
data_dumm.head()

,num_of_cosponsors,bill_char_counts,labels,congress_id_111th,congress_id_112th,congress_id_113th,congress_id_114th,congress_id_115th,intro_month_2,intro_month_3,...,sponsor_state_TX,sponsor_state_UT,sponsor_state_VA,sponsor_state_VI,sponsor_state_VT,sponsor_state_WA,sponsor_state_WI,sponsor_state_WV,sponsor_state_WY,session_2
0,3,5366,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,68574,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,24935,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,33853,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,23,3465,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [80]:
y = data_dumm.pop('labels').values

In [74]:
# ols = sm.OLS(y, X)
# results = ols.fit()
# results.summary()

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 123)

In [83]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
logreg.predict(X_test)

ValueError: Unknown label type: 'unknown'

In [ ]:
# passed_df = data[data['labels'] == 1]

# fig = plt.figure(figsize = (30, 12))
# ax = fig.add_subplot(111)
# ax.set_title('Number of Bills Introduced (red) and Passed (green) vs. Time', fontdict={'fontsize': 24})
# ax.hist(data['intro_date'], bins = 500, alpha = .5)
# ax.hist(passed_df['intro_date'], bins = 500, color = 'g')
# plt.show()

In [ ]:
passed_df = data[data['labels'] == 1]

fig = plt.figure(figsize = (30, 12))
ax = fig.add_subplot(111)
ax.set_title('Number of Bills Introduced (red) and Passed (green) vs. Time', fontdict={'fontsize': 24})
ax.hist(data['intro_date'], bins = 500, alpha = .5)
ax.hist(passed_df['intro_date'], bins = 1000, color = 'g')
ax.set_ylim(0, 400)
plt.show()